# Lancia Tuning Basic Calculators

In [1]:
using Unitful
using UnitfulUS
using Statistics

In [2]:
# Register jupyter notebook as a source of units 
# for new units we define
Unitful.register(Main);
@unit HP "HP" Horsepower 0.7456999*u"kW" false
@unit cc "cc" CubicCentimeter 1u"cm^3" false
@derived_dimension VolumeFlow  Unitful.𝐋^3 / Unitful.𝐓

### Lancia Specific Variables

In [3]:
# General
lancia_ccs = 1756u"cc";
lancia_cylinders = 4;
# Fueling
lancia_injector_maxf = 158u"cm^3/minute";
lancia_fuel_pump_maxp = 72u"psi";
# Boost
lancia_max_boost = 10u"psi";

### General Constants

In [4]:
stoic_afr_gas = 14.68;
mass_gas = 6.259u"lb/gal_us";
injector_pres_nom = 3u"bar";
# test the injector pressure convert to psi
uconvert(u"psi", injector_pres_nom);

43.51132131906277 psi

Simple conversion functions to get from `lbs/hr` to `cc/min` and back quickly

In [5]:
gas_lb_cc(lb) = u"cm^3/minute"(lb / mass_gas);
gas_cc_lb(cc) = u"gal_us/hr"(cc) * mass_gas;

# Injector Calculator

> BSFC = brake specific fuel consumption - effeciency measure per horsepower of engine. The Turbo and NA BSFC values are taken from Greg Banish's book. Currently taking the mean of the upper/lower bounds he specs, conservatively we might even want to take the max values.

## Current Injector Power Capacity
Calculate max power possible with our current 158cc injectors with a 20% margin for safety at stock pressure

In [6]:
"""
Get the maximum power a set of injectors can make with the given margin and turbo
"""
function max_power(inj_size::VolumeFlow, turbo::Bool=true, cylinders::Int=4, margin=0.2)
    BSFC_turbo = mean([0.55u"lb/(HP*hr)", 0.7u"lb/(HP*hr)"]);
    BSFC_na = mean([0.3u"lb/(HP*hr)", 0.5u"lb/(HP*hr)"]);
    # TODO: add margin
    fuel_cap_per_cylinder = inj_size |> gas_cc_lb;
    fuel_capacity = fuel_cap_per_cylinder * cylinders * (1-margin);
    max_power = turbo ? fuel_capacity / BSFC_turbo : fuel_capacity / BSFC_na;
    return max_power;
end

max_power

In [7]:
max_power(lancia_injector_maxf, true)

80.25463430004476 HP

## Calculate Injector Size for Power Goal
Given a power goal, what size injectors do we need to make this happen?

In [8]:
"""
Returns the approx fuel needs of an engine. Turbo engines run richer
and burn more fuel per power level therefore a turbo parameter is required.
"""
function fuel_needs(power::Unitful.Power, turbo::Bool=false)
    power_hp = uconvert(u"HP", power);
    if(turbo)
        BSFC_turbo = mean([0.55u"lb/(HP*hr)", 0.7u"lb/(HP*hr)"]);
        BSFC_turbo * power_hp;
    else
        BSFC_na = mean([0.3u"lb/(HP*hr)", 0.5u"lb/(HP*hr)"]);
        BSFC_na * power_hp;
    end
end

fuel_needs

In [9]:
"""
Calculates fuel needs per cylinder. Margin is recommended to be 20% or 0.2 for safety.
"""
function fuel_needs_per_cylinder(power::Unitful.Power, turbo::Bool=false, cylinders::Int=4, margin=0.2) 
    return fuel_needs(power, turbo) * (1+margin) / cylinders
end

fuel_needs_per_cylinder

Calculate minimum injector size in `cc/min` required to hit a goal power level. Mess with target power level to see what size injectors we need.

In [10]:
inj_size_min = fuel_needs_per_cylinder(150u"HP", true, lancia_cylinders) |> gas_lb_cc

283.4976471880492 cm^3 minute^-1

Test here to see what the calculator would recommend for the subaru. At 520BHP it's recommending ~1000cc injectors. Actually uses 1300ccs so has some extra capacity.

In [11]:
fuel_needs_per_cylinder(520u"HP", true, 4) |> gas_lb_cc

982.7918435852372 cm^3 minute^-1

## Fuel Pressure Change Calculation
If we ran the current injectors at an increased pressure, what kind of power could we make?

We also need to consider max fuel pump pressure. Fuel rail pressure plus boost  pressure needs to be less than max fuel pump pressure, so we can't go too high with fuel rail pressure.

In [12]:
" Calculates change in flow rate for injector from stock to given pressure"
flow_rate(pressure::Unitful.Pressure, nom_flow::VolumeFlow) = gas_cc_lb(nom_flow) * sqrt(pressure / injector_pres_nom) |> gas_lb_cc

flow_rate

In [13]:
lancia_flowrate_4bar = flow_rate(4u"bar", lancia_injector_maxf)
max_power(lancia_flowrate_4bar, true)

92.67006943369162 HP

In [14]:
lancia_flowrate_5bar = flow_rate(5u"bar", lancia_injector_maxf)
max_power(lancia_flowrate_5bar, true)

103.60828736667997 HP

In [15]:
5u"bar" |> u"psi"

72.51886886510462 psi

Based on fuel pump max pressure and boost pressure, our rail pressure is limited. Calculate max power given this restriction.

In [16]:
function max_rail_pressure(max_pump_pressure::Unitful.Pressure, max_boost::Unitful.Pressure)
    # TODO: lines will have some resistance, maybe add another safety factor here?
    return max_pump_pressure - max_boost;
end

max_rail_pressure (generic function with 1 method)

In [17]:
lancia_max_rail_pressure = max_rail_pressure(lancia_fuel_pump_maxp, lancia_max_boost)

62 psi

In [18]:
lancia_flowrate_max = flow_rate(lancia_max_rail_pressure, lancia_injector_maxf)
lancia_max_power = max_power(lancia_flowrate_max, true)

95.79984997301294 HP

end